In [1]:
import yaml

from linkml.utils.schema_builder import SchemaBuilder
from linkml_runtime.linkml_model import SlotDefinition


In [2]:
sb = SchemaBuilder()
sb.add_class("Person", slots=[SlotDefinition("family_name", range="string"), 
                              SlotDefinition("given_name", range="string"),
                              SlotDefinition("age_in_years", range="integer"),
                              SlotDefinition("height_in_cm", range="float"),
                              ])
sb.add_defaults()
print(yaml.dump(sb.as_dict(), sort_keys=False))


name: test-schema
id: http://example.org/test-schema
imports:
- linkml:types
prefixes:
  linkml: https://w3id.org/linkml/
  test_schema: http://example.org/test-schema/
default_prefix: test_schema
default_range: string
slots:
  family_name:
    range: string
  given_name:
    range: string
  age_in_years:
    range: integer
  height_in_cm:
    range: float
classes:
  Person:
    slots:
    - family_name
    - given_name
    - age_in_years
    - height_in_cm



In [3]:
from linkml_map.session import Session

session = Session()
session.set_source_schema(sb.as_dict())

In [7]:
# Transformer specification (in YAML)
session.set_object_transformer("""
class_derivations:
  Individual:
    populated_from: Person
    slot_derivations:
      family_name:
        populated_from: family_name
      given_name:
        populated_from: given_name
      age:
        populated_from: age_in_years
      height:
        populated_from: height_in_cm
""")

obj = {
        "given_name": "Jane",
        "family_name": "Doe",
        "age_in_years": 42,
        "height_in_cm": 180.0,
    }
session.transform(obj)

{'family_name': 'Doe', 'given_name': 'Jane', 'age': 42, 'height': 180.0}

In [8]:
from linkml_runtime.dumpers import yaml_dumper

print(yaml_dumper.dumps(session.target_schema))

name: test-schema-derived
id: http://example.org/test-schema-derived
imports:
- linkml:types
prefixes:
  linkml:
    prefix_prefix: linkml
    prefix_reference: https://w3id.org/linkml/
  test_schema:
    prefix_prefix: test_schema
    prefix_reference: http://example.org/test-schema/
default_prefix: http://example.org/test-schema-derived/
default_range: string
classes:
  Individual:
    name: Individual
    attributes:
      family_name:
        name: family_name
        domain_of:
        - Person
        range: string
      given_name:
        name: given_name
        domain_of:
        - Person
        range: string
      age:
        name: age
        domain_of:
        - Person
        range: integer
      height:
        name: height
        domain_of:
        - Person
        range: float



In [9]:
session.set_object_transformer("""
class_derivations:
  Individual:
    populated_from: Person
    slot_derivations:
      name:
        expr: "{given_name} + ' ' + {family_name}"
        description: Concatenating given and family names
          note this is a bad assumption for names in general,
          this is just for demonstration
      age_in_months:
        expr: age_in_years * 12
      height_in_meters:
        expr: height_in_cm / 100
""")

session.transform(obj)

{'name': 'Jane Doe', 'age_in_months': 504, 'height_in_meters': 1.8}

In [10]:
print(yaml_dumper.dumps(session.transform(obj)))

name: Jane Doe
age_in_months: 504
height_in_meters: 1.8

